In [1]:
import pandas as pd
from sklearn import datasets
from evidently import ColumnMapping

from evidently.test_suite import TestSuite
from evidently.test_preset import DataStabilityTestPreset

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

app_train = pd.read_csv('../CSV-Excels/P7_datasets/application_train.csv', index_col=[0])
app_test = pd.read_csv('../CSV-Excels/P7_datasets/application_test.csv', index_col=[0])

/home/kilian/anaconda3/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/kilian/anaconda3/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/kilian/anaconda3/lib/python3.9/site-packages/umap/distances.py:1086: NumbaDeprecationWarni

In [2]:
app_train.reset_index(inplace=True)
app_test.reset_index(inplace=True)

In [3]:
app_train.drop(columns=['TARGET', 'SK_ID_CURR'], inplace=True)
app_test.drop(columns=['SK_ID_CURR'], inplace=True)

In [4]:
app_test.describe()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,48744.000000,4.874400e+04,4.874400e+04,48720.000000,4.874400e+04,48744.000000,48744.000000,48744.000000,48744.000000,48744.000000,...,48744.000000,48744.0,48744.0,48744.0,42695.000000,42695.000000,42695.000000,42695.000000,42695.000000,42695.000000
mean,0.397054,1.784318e+05,5.167404e+05,29426.240209,4.626188e+05,0.021226,-16068.084605,67485.366322,-4967.652716,-3051.712949,...,0.001559,0.0,0.0,0.0,0.002108,0.001803,0.002787,0.009299,0.546902,1.983769
std,0.709047,1.015226e+05,3.653970e+05,16016.368315,3.367102e+05,0.014428,4325.900393,144348.507136,3552.612035,1569.276709,...,0.039456,0.0,0.0,0.0,0.046373,0.046132,0.054037,0.110924,0.693305,1.838873
min,0.000000,2.694150e+04,4.500000e+04,2295.000000,4.500000e+04,0.000253,-25195.000000,-17463.000000,-23722.000000,-6348.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.125000e+05,2.606400e+05,17973.000000,2.250000e+05,0.010006,-19637.000000,-2910.000000,-7459.250000,-4448.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.575000e+05,4.500000e+05,26199.000000,3.960000e+05,0.018850,-15785.000000,-1293.000000,-4490.000000,-3234.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,1.000000,2.250000e+05,6.750000e+05,37390.500000,6.300000e+05,0.028663,-12496.000000,-296.000000,-1901.000000,-1706.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000
max,20.000000,4.410000e+06,2.245500e+06,180576.000000,2.245500e+06,0.072508,-7338.000000,365243.000000,0.000000,0.000000,...,1.000000,0.0,0.0,0.0,2.000000,2.000000,2.000000,6.000000,7.000000,17.000000


In [5]:
app_train['FLAG_CONT_MOBILE'] = pd.to_numeric(app_train['FLAG_CONT_MOBILE'], errors='coerce')
# app_tra['FLAG_CONT_MOBILE'] = app_train['FLAG_CONT_MOBILE'].apply(pd.to_numeric)

In [6]:
app_test['FLAG_CONT_MOBILE'] = pd.to_numeric(app_test['FLAG_CONT_MOBILE'], errors='coerce')

In [7]:
app_train.iloc[:,23]
# app_train.columns.get_loc("FLAG_CONT_MOBILE")

0         1
1         1
2         1
3         1
4         1
         ..
307506    1
307507    1
307508    1
307509    1
307510    1
Name: FLAG_CONT_MOBILE, Length: 307511, dtype: int64

In [8]:
num_columns = app_train._get_numeric_data().columns.to_list()

In [9]:
column_mapping = ColumnMapping()
# column_mapping.categorical_features
column_mapping.numerical_features = ['FLAG_CONT_MOBILE']

In [10]:
data_stability= TestSuite(tests=[
    DataStabilityTestPreset(),
])
data_stability.run(current_data=app_test, reference_data=app_train, column_mapping=column_mapping)
data_stability 

In [11]:
data_drift_report = Report(metrics=[
    DataDriftPreset(),
])
data_drift_report.run(current_data=app_test, reference_data=app_train, column_mapping=column_mapping)
data_drift_report